# EDA: E-commerce Fraud Detection
This notebook contains exploratory data analysis of the e-commerce transaction data for fraud detection.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
sns.set_style('whitegrid')
%matplotlib inline

In [ ]:
# Load the data
data_dir = Path('../data/raw')
fraud_data = pd.read_csv(data_dir / 'Fraud_Data.csv')
ip_data = pd.read_csv(data_dir / 'IpAddress_to_Country.csv')

# Display basic info
print("Fraud Data Shape:", fraud_data.shape)
print("\nFraud Data Info:")
fraud_data.info()

In [ ]:
# Basic statistics
fraud_data.describe()

In [ ]:
# Check for missing values
fraud_data.isnull().sum()

In [ ]:
# Class distribution
class_dist = fraud_data['class'].value_counts(normalize=True)
plt.figure(figsize=(8, 6))
sns.barplot(x=class_dist.index, y=class_dist.values)
plt.title('Class Distribution')
plt.xlabel('Class (0: Legitimate, 1: Fraudulent)')
plt.ylabel('Percentage')
plt.show()